In [8]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
from datetime import datetime, timedelta

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}



## Buscando dados da tabela 'ml_fulfillment' para alimentar tabela 'fulfillment_stock'

In [ ]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM ml_fulfillment"

df = pd.read_sql(sql_query, conn)

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
print(df['ml_inventory_id'].value_counts())
print(len(df['ml_inventory_id'].unique()))

Até aqui:

596 é o total de ml_inventory_id únicos. Ou seja, temos 596 produtos únicos neste caso

In [ ]:
# Filtrando
valor_desejado = 'TNXB03680'
resultado = df.loc[df['ml_inventory_id'] == valor_desejado]
resultado

In [ ]:
df = df.drop_duplicates()
print(df.shape)
print(df['ml_inventory_id'].value_counts())
print(len(df['ml_inventory_id'].unique()))

In [ ]:
# Filtrando
valor_desejado = 'UIUW89800'
resultado = df.loc[df['ml_inventory_id'] == valor_desejado]
resultado

### Filtrando as colunas necessárias

In [ ]:
cols = ['ml_inventory_id', 'available_quantity', 'created_at']
db_fulfillment = df[cols].copy()
db_fulfillment

In [ ]:
duplicates = db_fulfillment.duplicated()

num_duplicates = duplicates.sum()

print("Número de linhas duplicadas:", num_duplicates)

duplicated_rows = db_fulfillment[duplicates]
print("Linhas duplicadas:")
print(duplicated_rows)



In [ ]:
db_fulfillment = db_fulfillment.drop_duplicates()
print(db_fulfillment.shape)
# print(db_fulfillment['ml_inventory_id'].value_counts())
# print(len(db_fulfillment['ml_inventory_id'].unique()))

In [ ]:
db_fulfillment

In [ ]:
db_fulfillment.info()

### Enviando Dados para a tabela 'fulfillment_stock'

In [ ]:

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in db_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
        VALUES (%s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['available_quantity'], row['created_at']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in db_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
        VALUES (%s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['available_quantity'], row['created_at']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


## Buscando dados de 'fulfillment_stock'

In [104]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM fulfillment_stock"

df = pd.read_sql(sql_query, conn)
df.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_20760\4273217703.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


(18476, 3)

In [105]:
## verificando quantidade de dias cobertos pela tabela 'fulfillment_stock' ##
df.shape[0] / 596

31.0

In [106]:
## Criando coluna de data ##

df['data'] = df['created_at'].dt.date

df

,ml_inventory_id,available_quantity,created_at,data
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09
3,ZQIV51979,0,2023-11-09 16:33:03.451375,2023-11-09
4,RBUB53192,15,2023-11-09 16:33:03.451375,2023-11-09
...,...,...,...,...
18471,RKDM61361,13,2023-10-10 01:00:00.000000,2023-10-10
18472,PPWL59058,9,2023-10-10 01:00:00.000000,2023-10-10
18473,RXXA58942,141,2023-10-10 01:00:00.000000,2023-10-10
18474,POTW78210,2,2023-10-10 01:00:00.000000,2023-10-10


In [109]:
## verificando onde df_filtrado2['available_quantity'] = 0 ##

valor_desejado = 0  
linhas_selecionadas = df[df['available_quantity'] == valor_desejado]

linhas_selecionadas.head(3)

,ml_inventory_id,available_quantity,created_at,data
3,ZQIV51979,0,2023-11-09 16:33:03.451375,2023-11-09
5,FXMN50419,0,2023-11-09 16:33:03.451375,2023-11-09
6,TDFV51283,0,2023-11-09 16:33:03.451375,2023-11-09


In [110]:
## verificando ml_inventory_id ##

valor_desejado = 'ZQIV51979'  
linhas_selecionadas = df[df['ml_inventory_id'] == valor_desejado]

linhas_selecionadas.head(3)

,ml_inventory_id,available_quantity,created_at,data
3,ZQIV51979,0,2023-11-09 16:33:03.451375,2023-11-09
599,ZQIV51979,0,2023-11-08 01:00:00.000000,2023-11-08
1195,ZQIV51979,0,2023-11-07 01:00:00.000000,2023-11-07


In [144]:
## Mudando valores de 'available_quantity', onde available_quantity=0, available_quantity=1 

df.loc[df['available_quantity'] == 0, 'available_quantity'] = 1
df.head(4)

,ml_inventory_id,available_quantity,created_at,data
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09


In [146]:
## verificando ml_inventory_id ##

valor_desejado = 'ZQIV51979'  
linhas_selecionadas = df[df['ml_inventory_id'] == valor_desejado]

linhas_selecionadas.head(3)

,ml_inventory_id,available_quantity,created_at,data
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09
599,ZQIV51979,1,2023-11-08 01:00:00.000000,2023-11-08
1195,ZQIV51979,1,2023-11-07 01:00:00.000000,2023-11-07


### Filtrando do DF por dias

In [189]:
# não calcular o dia atual
# se hoje 09, calcular d x até 08
days = 29

data_atual = datetime.now().date()  # Extrai a data do datetime de data atual
data_inicio = (data_atual - timedelta(days=days))  # Extrai a data do datetime da data de início

print(data_inicio)
print(data_atual)

# Filtra para incluir apenas as linhas dentro do período desejado
df_filtrado = df[(df['data'] >= data_inicio) & (df['data'] <= data_atual)]


2023-10-11
2023-11-09


In [190]:
df_filtrado.shape

(17880, 4)

In [191]:
df_filtrado['ml_inventory_id'].value_counts()

ml_inventory_id
JFGN34621    30
FTGG16520    30
EDHV29406    30
GUSW29256    30
UCDR32919    30
             ..
PNST71108    30
GLMK71067    30
VFWO70372    30
EDGX70669    30
UCQQ19395    30
Name: count, Length: 596, dtype: int64

In [192]:
df_filtrado.head(4)

,ml_inventory_id,available_quantity,created_at,data
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09


In [193]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_filtrado = df_filtrado.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_filtrado.head(4)

,ml_inventory_id,available_quantity,created_at,data,has_stock
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09,True
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09,True
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09,True
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09,True


In [194]:
## Contando dias em que produto esteve disponível ##
days_available = df_filtrado.groupby('ml_inventory_id')['has_stock'].sum().reset_index()

# Renomeie a coluna resultante para 'days_available'
days_available = days_available.rename(columns={'has_stock': 'days_available'})

df_filtrado = df_filtrado.merge(days_available, on='ml_inventory_id', how='left')
df_filtrado

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09,True,30
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09,True,30
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09,True,30
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09,True,30
4,RBUB53192,15,2023-11-09 16:33:03.451375,2023-11-09,True,30
...,...,...,...,...,...,...
17875,RKDM61361,13,2023-10-11 01:00:00.000000,2023-10-11,True,30
17876,PPWL59058,9,2023-10-11 01:00:00.000000,2023-10-11,True,30
17877,RXXA58942,141,2023-10-11 01:00:00.000000,2023-10-11,True,30
17878,POTW78210,2,2023-10-11 01:00:00.000000,2023-10-11,True,30


In [195]:
# verificando dados
valor_desejado = 'ZQIV51979'  
linhas_selecionadas = df_filtrado[df_filtrado['ml_inventory_id'] == valor_desejado]

linhas_selecionadas

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09,True,30
599,ZQIV51979,1,2023-11-08 01:00:00.000000,2023-11-08,True,30
1195,ZQIV51979,1,2023-11-07 01:00:00.000000,2023-11-07,True,30
1791,ZQIV51979,1,2023-11-06 01:00:00.000000,2023-11-06,True,30
2387,ZQIV51979,1,2023-11-05 01:00:00.000000,2023-11-05,True,30
2983,ZQIV51979,1,2023-11-04 01:00:00.000000,2023-11-04,True,30
3579,ZQIV51979,1,2023-11-03 01:00:00.000000,2023-11-03,True,30
4175,ZQIV51979,1,2023-11-02 01:00:00.000000,2023-11-02,True,30
4771,ZQIV51979,1,2023-11-01 01:00:00.000000,2023-11-01,True,30
5367,ZQIV51979,1,2023-10-31 01:00:00.000000,2023-10-31,True,30


In [196]:
## Calculando média  de available_quantity / período de dias se has_stock = True ##
df_filtrado['media_available_quantity'] = df_filtrado.groupby('ml_inventory_id')['available_quantity'].transform('mean')
df_filtrado['media_available_quantity'] = df_filtrado[df_filtrado['has_stock']].groupby('ml_inventory_id')['available_quantity'].transform('mean')

df_filtrado.head(4)

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,media_available_quantity
0,JFGN34621,71,2023-11-09 16:33:03.451375,2023-11-09,True,30,71.0
1,ERIM51807,34,2023-11-09 16:33:03.451375,2023-11-09,True,30,34.0
2,HVCS52763,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
3,ZQIV51979,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0


In [203]:
df_filtrado['ml_inventory_id'].value_counts()

ml_inventory_id
JFGN34621    30
FTGG16520    30
EDHV29406    30
GUSW29256    30
UCDR32919    30
             ..
PNST71108    30
GLMK71067    30
VFWO70372    30
EDGX70669    30
UCQQ19395    30
Name: count, Length: 596, dtype: int64

In [205]:
## pegando as linhas mais recentes, ultimo dia ##

indices_linhas_recentes = df_filtrado.groupby('ml_inventory_id')['created_at'].idxmax()
# indices_linhas_recentes = df_filtrado.groupby('ml_inventory_id')['data'].idxmax()

df_hoje = df_filtrado.loc[indices_linhas_recentes]
df_hoje

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,media_available_quantity
34,ABCB20467,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
165,ABCB48480,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
287,ACBY18022,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
365,ADJX46855,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
388,ADRN17806,1,2023-11-09 16:33:03.451375,2023-11-09,True,30,1.0
...,...,...,...,...,...,...,...
573,ZXJO91694,7,2023-11-09 16:33:03.451375,2023-11-09,True,30,7.0
45,ZZGN21450,3,2023-11-09 16:33:03.451375,2023-11-09,True,30,3.0
481,ZZQL21216,6,2023-11-09 16:33:03.451375,2023-11-09,True,30,6.0
54,ZZRV30730,7,2023-11-09 16:33:03.451375,2023-11-09,True,30,7.0


In [206]:
# verificando dados

valor_desejado = 'JFGN34621'  
linhas_selecionadas = df_hoje[df_hoje['ml_inventory_id'] == valor_desejado]

linhas_selecionadas

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,media_available_quantity
0,JFGN34621,42,2023-11-09 16:33:03.451375,2023-11-09,True,30,71.0


In [178]:
# calculando a quantidade de produtos a enviar
import math
df_filtrado2['send_fulfillment'] = df_filtrado2['media_available_quantity'] - df_filtrado2['available_quantity']
df_filtrado2['send_fulfillment'] = df_filtrado2['send_fulfillment'].apply(lambda x: math.ceil(x)) # arredonda para cima


# verificando dados

valor_desejado = 'JFGN34621'  
linhas_selecionadas = df_filtrado2[df_filtrado2['ml_inventory_id'] == valor_desejado]

linhas_selecionadas

,ml_inventory_id,available_quantity,created_at,data,has_stock,media_available_quantity,send_fulfillment
0,JFGN34621,42,2023-11-09 16:33:03.451375,2023-11-09,True,70.033333,29


In [179]:
df_filtrado2

,ml_inventory_id,available_quantity,created_at,data,has_stock,media_available_quantity,send_fulfillment
34,ABCB20467,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0
165,ABCB48480,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0
287,ACBY18022,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0
365,ADJX46855,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0
388,ADRN17806,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0
...,...,...,...,...,...,...,...
573,ZXJO91694,7,2023-11-09 16:33:03.451375,2023-11-09,True,7.0,0
45,ZZGN21450,3,2023-11-09 16:33:03.451375,2023-11-09,True,3.0,0
481,ZZQL21216,6,2023-11-09 16:33:03.451375,2023-11-09,True,6.0,0
54,ZZRV30730,7,2023-11-09 16:33:03.451375,2023-11-09,True,7.0,0


In [183]:
df_filtrado2['days_available'] = df_filtrado2['has_stock'].sum()
df_filtrado2

,ml_inventory_id,available_quantity,created_at,data,has_stock,media_available_quantity,send_fulfillment,days_available
34,ABCB20467,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0,596
165,ABCB48480,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0,596
287,ACBY18022,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0,596
365,ADJX46855,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0,596
388,ADRN17806,1,2023-11-09 16:33:03.451375,2023-11-09,True,1.0,0,596
...,...,...,...,...,...,...,...,...
573,ZXJO91694,7,2023-11-09 16:33:03.451375,2023-11-09,True,7.0,0,596
45,ZZGN21450,3,2023-11-09 16:33:03.451375,2023-11-09,True,3.0,0,596
481,ZZQL21216,6,2023-11-09 16:33:03.451375,2023-11-09,True,6.0,0,596
54,ZZRV30730,7,2023-11-09 16:33:03.451375,2023-11-09,True,7.0,0,596
